# langchain学习

### 一、切割文档用于制作知识库

In [ ]:
from langchain_community.document_loaders import UnstructuredFileLoader
# 旧版用法
# from langchain.document_loaders import UnstructuredFileLoader

from langchain.text_splitter import RecursiveCharacterTextSplitter

# 导入文本
loader = UnstructuredFileLoader("西游记.txt")
# 将文本转成 Document 对象
data = loader.load()
print(f'documents:{len(data)}')

# 初始化加载器
text_splitter = RecursiveCharacterTextSplitter(chunk_size=100, chunk_overlap=0)
# 切割加载的 document
split_docs = text_splitter.split_documents(data)
print("split_docs size:",len(split_docs))
print(split_docs)

### 二、设置emmbeding层，用于数据的向量化，存入数据库

In [6]:
from langchain_community.embeddings import DashScopeEmbeddings
embedding_name = r"text-embedding-v1"
ds_embeddings = DashScopeEmbeddings(model=embedding_name)


### 三、保存向量到数据库

In [9]:
from langchain_community.vectorstores import Chroma
import sentence_transformers

# # 初始化数据库
# db = Chroma.from_documents(split_docs, ds_embeddings,persist_directory="./chroma/news_test")
# # 持久化
# db.persist()
# 对数据进行加载
db = Chroma(persist_directory="./chroma/news_test", embedding_function=ds_embeddings)



### 四、测试知识库是否可用

In [12]:
question = "三打白骨精"
# 寻找四个相似的样本
similarDocs = db.similarity_search(question,k=4)
print(similarDocs)


[Document(page_content='为至宝，怎肯轻与你这粉骷髅。”他两个散言碎语的，直斗到更深，唐长老全不动念。那女怪扯扯拉拉的不放，这师父只是老老成成的不肯。直缠到有半夜时候，把那怪弄得恼了，叫：“小的们，拿绳来！”可怜将一个心爱的人', metadata={'source': '西游记.txt'}), Document(page_content='的，若不做个威风，他怎肯服降哩？”陡然间，就狐假虎威，红须倒竖，血发朝天，眼睛迸裂，大喝一声道：“带那和尚进来！”众妖们，大家响响的答应了一声“是！”就把三藏望里面只是一推。这是既在矮檐下，怎敢不低头', metadata={'source': '西游记.txt'}), Document(page_content='！”那妖精就举铲来敌。这一场，从天井中斗出前门。看他三僧攒一怪！好杀：', metadata={'source': '西游记.txt'}), Document(page_content='打杀了，拖在门口骂战哩。”那老妖闻言，愈加烦恼道：“这厮却也无知！我倒不曾吃他师父，他转打杀我家先锋，可恨！可恨！”叫：“取披挂来。我也只闻得讲甚么孙行者，等我出去，看是个甚么九头八尾的和尚，拿他进来', metadata={'source': '西游记.txt'})]


### 五、调用大模型来结合数据库来问答

In [23]:

from langchain.chains import RetrievalQA
from langchain_community.llms import Tongyi

retriever = db.as_retriever()
qa = RetrievalQA.from_chain_type(llm=Tongyi(), retriever=retriever)

query = "三打白骨精是第几章？"
print(qa.run(query))


 三打白骨精是上卷的第二十七回。


### 注：
#### 教程采用了通义千问的emmbeding层和大模型，需要到阿里云上根据教程设置调用的key
#### 链接：https://help.aliyun.com/zh/dashscope/developer-reference/activate-dashscope-and-create-an-api-key?spm=a2c4g.11186623.0.i21